# 실습 과제

1. 데이터 RDD 생성 map()
2. collect()
3. blue mnms만 꺼내기
4. 전체 합계 구하기
5. 도시별 합계 구하기

In [1]:
import os
from pyspark import SparkConf, SparkContext

In [2]:
# 스파크 환경 설정, 객체 생성
conf = SparkConf().setMaster("local").setAppName("mnm-rdd-counts")
spark = SparkContext(conf=conf).getOrCreate()

24/12/03 15:33:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/03 15:33:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/03 15:33:25 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
directory = os.path.join(os.getcwd(), "data")
filename = "mnm_dataset.csv"
filepath = os.path.join(directory, filename)

In [4]:
lines = spark.textFile("file:///"+filepath.replace("\\", "/"))
lines

file:////home/lab06/src/data/mnm_dataset.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [6]:
lines.collect()

['State,Color,Count',
 'TX,Red,20',
 'NV,Blue,66',
 'CO,Blue,79',
 'OR,Blue,71',
 'WA,Yellow,93',
 'WY,Blue,16',
 'CA,Yellow,53',
 'WA,Green,60',
 'OR,Green,71',
 'TX,Green,68',
 'NV,Green,59',
 'AZ,Brown,95',
 'WA,Yellow,20',
 'AZ,Blue,75',
 'OR,Brown,72',
 'NV,Red,98',
 'WY,Orange,45',
 'CO,Blue,52',
 'TX,Brown,94',
 'CO,Red,82',
 'CO,Red,12',
 'CO,Red,17',
 'OR,Green,16',
 'AZ,Green,46',
 'NV,Red,43',
 'NM,Yellow,15',
 'WA,Red,12',
 'OR,Green,13',
 'CO,Blue,95',
 'WY,Red,63',
 'TX,Orange,63',
 'WY,Yellow,48',
 'OR,Green,95',
 'WA,Red,75',
 'CO,Orange,93',
 'NV,Orange,10',
 'WY,Green,15',
 'WA,Green,99',
 'CO,Blue,98',
 'CA,Green,86',
 'UT,Red,92',
 'AZ,Brown,16',
 'CA,Red,100',
 'UT,Red,77',
 'TX,Yellow,29',
 'WA,Orange,73',
 'WY,Yellow,56',
 'WA,Yellow,19',
 'WY,Yellow,18',
 'AZ,Orange,49',
 'WA,Red,83',
 'AZ,Red,59',
 'CA,Blue,13',
 'CO,Brown,53',
 'CA,Red,23',
 'CA,Yellow,49',
 'NV,Blue,50',
 'CO,Red,81',
 'UT,Green,62',
 'NM,Green,38',
 'NV,Yellow,77',
 'TX,Blue,49',
 'WA,Brown,

In [7]:
header = lines.first()
filtered_lines = lines.filter(lambda row : row != header)

In [8]:
filtered_lines.take(5)

['TX,Red,20', 'NV,Blue,66', 'CO,Blue,79', 'OR,Blue,71', 'WA,Yellow,93']

In [15]:
def parse_line(line):
    fields = line.split(",")
    state = fields[0]
    color = fields[1]
    count = int(fields[2])
    return (state, color, count)

In [16]:
parsed_rdd = filtered_lines.map(parse_line)

In [23]:
# Blue M&Ms만 필터링
blue_mnms = filtered_lines.filter(lambda line: line.split(",")[1] == "Blue")

# Blue M&Ms 출력
print("Blue M&Ms:")
for item in blue_mnms.collect():
    print(item)

Blue M&Ms:
NV,Blue,66
CO,Blue,79
OR,Blue,71
WY,Blue,16
AZ,Blue,75
CO,Blue,52
CO,Blue,95
CO,Blue,98
CA,Blue,13
NV,Blue,50
TX,Blue,49
CA,Blue,34
UT,Blue,97
AZ,Blue,59
OR,Blue,29
OR,Blue,54
CO,Blue,52
WA,Blue,59
WY,Blue,59
WY,Blue,53
WA,Blue,42
WA,Blue,37
NV,Blue,49
NM,Blue,15
CA,Blue,39
UT,Blue,48
AZ,Blue,82
CO,Blue,14
WA,Blue,18
CO,Blue,89
TX,Blue,80
WA,Blue,84
NM,Blue,46
AZ,Blue,16
TX,Blue,68
CA,Blue,99
WY,Blue,80
CO,Blue,44
CO,Blue,90
TX,Blue,19
CA,Blue,65
TX,Blue,98
NM,Blue,98
AZ,Blue,24
NM,Blue,70
OR,Blue,96
OR,Blue,37
WY,Blue,33
WA,Blue,46
NM,Blue,52
OR,Blue,20
CO,Blue,27
NM,Blue,82
OR,Blue,74
CA,Blue,43
UT,Blue,21
WA,Blue,62
NM,Blue,70
WA,Blue,37
AZ,Blue,19
UT,Blue,94
NM,Blue,81
TX,Blue,41
NV,Blue,89
TX,Blue,76
TX,Blue,59
OR,Blue,95
UT,Blue,65
WY,Blue,35
WY,Blue,24
WA,Blue,14
CO,Blue,27
CA,Blue,23
CO,Blue,17
CO,Blue,22
CA,Blue,99
NM,Blue,13
WY,Blue,57
TX,Blue,60
TX,Blue,91
WY,Blue,62
CA,Blue,89
WA,Blue,42
NM,Blue,21
CA,Blue,65
AZ,Blue,38
NV,Blue,12
OR,Blue,89
WA,Blue,47
UT,Blue,19

In [24]:
# 전체 M&M 개수 구하기
total_mnm_count = filtered_lines.map(lambda line: int(line.split(",")[2])).sum()
print("전체 M&M 개수:", total_mnm_count)

전체 M&M 개수: 5500035


In [27]:
# 주별 합계 및 개수 계산
state_counts = filtered_lines.map(lambda line: (line.split(",")[0], int(line.split(",")[2])))

# 주별 리뷰 합계 및 개수 계산
state_review_count = state_counts.mapValues(lambda x: (x, 1))
reduced_rdd = state_review_count.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

# 주별 합계와 평균 계산
state_totals_and_averages = reduced_rdd.mapValues(lambda x: (x[0], x[0] / x[1]))

# 결과 출력
print("주별 M&M 합계 및 평균:")
for item in state_totals_and_averages.collect():
    state, (total, average) = item
    print(f"{state}: 합계 = {total}, 평균 = {average}")

주별 M&M 합계 및 평균:
TX: 합계 = 556493, 평균 = 55.25148927720413
NV: 합계 = 548477, 평균 = 54.71092269326683
CO: 합계 = 556302, 평균 = 55.347925579544324
OR: 합계 = 542169, 평균 = 54.74795516510149
WA: 합계 = 557227, 평균 = 55.362841530054645
WY: 합계 = 538975, 평균 = 55.09865058270292
CA: 합계 = 561184, 평균 = 55.21290830381739
AZ: 합계 = 546812, 평균 = 54.67573242675732
NM: 합계 = 553454, 평균 = 55.07004975124378
UT: 합계 = 538942, 평균 = 54.51567873760874


In [25]:
# 주 x 색상별 합계 및 개수 계산
state_color_counts = filtered_lines.map(lambda line: ((line.split(",")[0], line.split(",")[1]), int(line.split(",")[2])))
state_color_review_count = state_color_counts.mapValues(lambda x: (x, 1))
state_color_reduced_rdd = state_color_review_count.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

# 주 x 색상별 합계와 평균 계산
state_color_totals_and_averages = state_color_reduced_rdd.mapValues(lambda x: (x[0], x[0] / x[1]))

# 결과 출력
print("주 x 색상별 M&M 합계 및 평균:")
for item in state_color_totals_and_averages.collect():
    (state, color), (total, average) = item
    print(f"{state}, {color}: 합계 = {total}, 평균 = {average}")

주 x 색상별 M&M 합계 및 평균:
TX, Red: 합계 = 95404, 평균 = 55.306666666666665
NV, Blue: 합계 = 90003, 평균 = 53.797369994022716
CO, Blue: 합계 = 93412, 평균 = 55.11032448377581
OR, Blue: 합계 = 90526, 평균 = 54.99756986634265
WA, Yellow: 합계 = 92920, 평균 = 55.8749248346362
WY, Blue: 합계 = 91002, 평균 = 54.68870192307692
CA, Yellow: 합계 = 100956, 평균 = 55.8693967902601
WA, Green: 합계 = 96486, 평균 = 54.23608768971332
OR, Green: 합계 = 89578, 평균 = 54.821297429620564
TX, Green: 합계 = 95753, 평균 = 55.12550374208405
NV, Green: 합계 = 91331, 평균 = 53.78739693757362
AZ, Brown: 합계 = 92287, 평균 = 54.350412249705535
AZ, Blue: 합계 = 89971, 평균 = 54.99449877750611
OR, Brown: 합계 = 89136, 평균 = 54.98827884022209
NV, Red: 합계 = 89346, 평균 = 55.4944099378882
WY, Orange: 합계 = 87956, 평균 = 55.144827586206894
TX, Brown: 합계 = 90736, 평균 = 55.29311395490554
CO, Red: 합계 = 89465, 평균 = 55.089285714285715
AZ, Green: 합계 = 91882, 평균 = 54.82219570405728
NM, Yellow: 합계 = 92747, 평균 = 54.94490521327014
WA, Red: 합계 = 93332, 평균 = 55.85397965290245
WY, Red: 합계 = 9176

In [17]:
# Blue M&Ms만 필터링
blue_mnms = parsed_rdd.filter(lambda x: x[1] == "Blue")

print("Blue M&Ms:")
for item in blue_mnms.collect():
    print(item)

Blue M&Ms:
('NV', 'Blue', 66)
('CO', 'Blue', 79)
('OR', 'Blue', 71)
('WY', 'Blue', 16)
('AZ', 'Blue', 75)
('CO', 'Blue', 52)
('CO', 'Blue', 95)
('CO', 'Blue', 98)
('CA', 'Blue', 13)
('NV', 'Blue', 50)
('TX', 'Blue', 49)
('CA', 'Blue', 34)
('UT', 'Blue', 97)
('AZ', 'Blue', 59)
('OR', 'Blue', 29)
('OR', 'Blue', 54)
('CO', 'Blue', 52)
('WA', 'Blue', 59)
('WY', 'Blue', 59)
('WY', 'Blue', 53)
('WA', 'Blue', 42)
('WA', 'Blue', 37)
('NV', 'Blue', 49)
('NM', 'Blue', 15)
('CA', 'Blue', 39)
('UT', 'Blue', 48)
('AZ', 'Blue', 82)
('CO', 'Blue', 14)
('WA', 'Blue', 18)
('CO', 'Blue', 89)
('TX', 'Blue', 80)
('WA', 'Blue', 84)
('NM', 'Blue', 46)
('AZ', 'Blue', 16)
('TX', 'Blue', 68)
('CA', 'Blue', 99)
('WY', 'Blue', 80)
('CO', 'Blue', 44)
('CO', 'Blue', 90)
('TX', 'Blue', 19)
('CA', 'Blue', 65)
('TX', 'Blue', 98)
('NM', 'Blue', 98)
('AZ', 'Blue', 24)
('NM', 'Blue', 70)
('OR', 'Blue', 96)
('OR', 'Blue', 37)
('WY', 'Blue', 33)
('WA', 'Blue', 46)
('NM', 'Blue', 52)
('OR', 'Blue', 20)
('CO', 'Blue', 27)
(

In [18]:
# 전체 M&M 개수 구하기
total_mnm_count = parsed_rdd.map(lambda x: x[2]).sum()
print("전체 M&M 개수:", total_mnm_count)

전체 M&M 개수: 5500035


In [20]:
# 전체 M&M 개수 구하기
total_mnm_count = parsed_rdd.map(lambda x: x[2]).sum()
print("전체 M&M 개수:", total_mnm_count)

# 주별 합계 및 개수 구하기
state_counts = parsed_rdd.map(lambda x: (x[0], x[2]))

# 주별 리뷰 합계 및 개수 계산
state_review_count = state_counts.mapValues(lambda x: (x, 1))
reduced_rdd = state_review_count.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

# 주별 합계 및 평균 계산
state_totals_and_averages = reduced_rdd.mapValues(lambda x: (x[0], x[0] / x[1]))

# 결과 출력
print("주별 M&M 합계 및 평균:")
for item in state_totals_and_averages.collect():
    state, (total, average) = item
    print(f"{state}: 합계 = {total}, 평균 = {average}")

전체 M&M 개수: 5500035
주별 M&M 합계 및 평균:
TX: 합계 = 556493, 평균 = 55.25148927720413
NV: 합계 = 548477, 평균 = 54.71092269326683
CO: 합계 = 556302, 평균 = 55.347925579544324
OR: 합계 = 542169, 평균 = 54.74795516510149
WA: 합계 = 557227, 평균 = 55.362841530054645
WY: 합계 = 538975, 평균 = 55.09865058270292
CA: 합계 = 561184, 평균 = 55.21290830381739
AZ: 합계 = 546812, 평균 = 54.67573242675732
NM: 합계 = 553454, 평균 = 55.07004975124378
UT: 합계 = 538942, 평균 = 54.51567873760874


In [22]:
# 주 x 색상별 합계 및 개수 계산
state_color_counts = parsed_rdd.map(lambda x: ((x[0], x[1]), x[2]))
state_color_review_count = state_color_counts.mapValues(lambda x: (x, 1))
state_color_reduced_rdd = state_color_review_count.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

# 주 x 색상별 합계와 평균 계산
state_color_totals_and_averages = state_color_reduced_rdd.mapValues(lambda x: (x[0], x[0] / x[1]))

# 결과 출력
print("주 x 색상별 M&M 합계 및 평균:")
for item in state_color_totals_and_averages.collect():b
    (state, color), (total, average) = item
    print(f"{state}, {color}: 합계 = {total}, 평균 = {average}")


주 x 색상별 M&M 합계 및 평균:
TX, Red: 합계 = 95404, 평균 = 55.306666666666665
NV, Blue: 합계 = 90003, 평균 = 53.797369994022716
CO, Blue: 합계 = 93412, 평균 = 55.11032448377581
OR, Blue: 합계 = 90526, 평균 = 54.99756986634265
WA, Yellow: 합계 = 92920, 평균 = 55.8749248346362
WY, Blue: 합계 = 91002, 평균 = 54.68870192307692
CA, Yellow: 합계 = 100956, 평균 = 55.8693967902601
WA, Green: 합계 = 96486, 평균 = 54.23608768971332
OR, Green: 합계 = 89578, 평균 = 54.821297429620564
TX, Green: 합계 = 95753, 평균 = 55.12550374208405
NV, Green: 합계 = 91331, 평균 = 53.78739693757362
AZ, Brown: 합계 = 92287, 평균 = 54.350412249705535
AZ, Blue: 합계 = 89971, 평균 = 54.99449877750611
OR, Brown: 합계 = 89136, 평균 = 54.98827884022209
NV, Red: 합계 = 89346, 평균 = 55.4944099378882
WY, Orange: 합계 = 87956, 평균 = 55.144827586206894
TX, Brown: 합계 = 90736, 평균 = 55.29311395490554
CO, Red: 합계 = 89465, 평균 = 55.089285714285715
AZ, Green: 합계 = 91882, 평균 = 54.82219570405728
NM, Yellow: 합계 = 92747, 평균 = 54.94490521327014
WA, Red: 합계 = 93332, 평균 = 55.85397965290245
WY, Red: 합계 = 9176

In [28]:
spark.stop()